In [1]:
# Jupyter Notebookのセルで先頭に!をつけると、シェルコマンドを実行できます
!pip install biopython pandas requests googletrans-py slack_sdk jupyter

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
  Using cached fqdn-1.5.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached isoduration-20.11.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached uri_template-1.3.0-py3-none-any.whl.metadata (8.8 kB)
  Using cached webcolors-24.11.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached arrow-1.3.0-py3-none-any.whl.metadata (7.5 kB)
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 41.3 MB/s eta 

  DEPRECATION: Building 'googletrans-py' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'googletrans-py'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
import os
import pandas as pd
import requests
from time import sleep
from Bio import Entrez
from googletrans import Translator
from slack_sdk.webhook import WebhookClient

# --- ★ ユーザー設定部分 ---

# 1. NCBI Entrezへの登録メールアドレス (必須)
# PubMedのAPIを利用するために、NCBIに自分の身元を伝えるメールアドレスが必要です。
Entrez.email = "gakukibou@gmail.com"  # ★ 必ずご自身のメールアドレスに書き換えてください

# 2. PubMedでの検索キーワード
# PubMedの検索窓で使うのと同じ形式で記述できます。
# 例：「(癌 OR 腫瘍) AND 免疫療法 AND "last 1 year"[Filter]」
SEARCH_TERM = '(("llm"[Title/Abstract] OR "large language model"[Title/Abstract])) AND "medical"[Title/Abstract]'

# 3. 検索する論文の最大数
MAX_RESULTS = 20

# 4. 各種ファイルの保存先
CSV_FILE_PATH = 'pubmed_論文リスト.csv'
# PDFダウンロードはPubMedでは直接できないため、今回はURLの保存までとします

# 5. SlackのWebhook URL
SLACK_WEBHOOK_URL = 'https://hooks.slack.com/services/...' # ★ 必ずご自身のSlack Webhook URLに書き換えてください

In [3]:
def translate_text(text, dest_lang='ja'):
    """テキストを翻訳する関数"""
    if not text:
        return ""
    try:
        translator = Translator()
        sleep(1) # API制限を避けるための待機
        translated = translator.translate(text, dest=dest_lang)
        return translated.text
    except Exception as e:
        print(f"翻訳中にエラーが発生しました: {e}")
        return "(翻訳失敗)"

def send_slack_notification(article_info):
    """Slackに新着論文の情報を通知する関数"""
    try:
        webhook = WebhookClient(SLACK_WEBHOOK_URL)
        title = article_info['title']
        authors = article_info['authors']
        journal = article_info['journal']
        pub_year = article_info['pub_year']
        url = article_info['url']
        abstract_ja = article_info['abstract_ja']
        
        message = f"""
        🔔 *PubMed新着論文が見つかりました！*
        
        *タイトル*: {title}
        *雑誌名*: {journal} ({pub_year}年)
        *著者*: {authors}
        *URL*: <{url}|PubMedで見る>
        
        *要約（日本語）*:
        ```{abstract_ja}```
        """
        response = webhook.send(text=message)
        if response.status_code == 200:
            print("Slackに通知を送信しました。")
        else:
            print(f"Slack通知に失敗しました (Status: {response.status_code})")
    except Exception as e:
        print(f"Slack通知中にエラーが発生しました: {e}")

In [4]:
def main_pubmed_search():
    """PubMedで論文を検索し、新規論文の処理を行う関数"""
    
    # 既存のCSVを読み込み、処理済みの論文ID (PMID) を取得
    try:
        df_existing = pd.read_csv(CSV_FILE_PATH)
        existing_ids = set(df_existing['pmid'].astype(str))
        print(f"既存の論文リストを読み込みました。{len(existing_ids)}件が登録済みです。")
    except FileNotFoundError:
        df_existing = pd.DataFrame()
        existing_ids = set()
        print("新規に論文リストを作成します。")

    # 1. PubMed ID (PMID) のリストを取得
    print(f"PubMedで検索中... (検索語: {SEARCH_TERM})")
    handle = Entrez.esearch(db="pubmed", term=SEARCH_TERM, retmax=MAX_RESULTS, sort="pub date")
    record = Entrez.read(handle)
    handle.close()
    id_list = record["IdList"]
    
    if not id_list:
        print("検索結果が0件でした。")
        return

    # 2. PMIDを使って各論文の詳細情報を取得
    print(f"{len(id_list)}件の論文情報を取得します。")
    handle = Entrez.efetch(db="pubmed", id=id_list, rettype="medline", retmode="xml")
    records = Entrez.read(handle)['PubmedArticle']
    handle.close()

    new_articles = []
    for record in records:
        # MedlineCitationから情報を取得
        citation = record['MedlineCitation']
        pmid = str(citation['PMID'])
        
        if pmid not in existing_ids:
            print(f"\n--- 新規論文を処理中 (PMID: {pmid}) ---")
            
            article = citation['Article']
            
            # 各情報を取得 (存在しない場合もあるため.get()で安全にアクセス)
            title = article.get('ArticleTitle', 'N/A')
            journal_info = article.get('Journal', {})
            journal = journal_info.get('Title', 'N/A')
            pub_date = journal_info.get('JournalIssue', {}).get('PubDate', {})
            pub_year = pub_date.get('Year', pub_date.get('MedlineDate', 'N/A').split(' ')[0])
            
            # 著者はリスト形式なので結合
            author_list = article.get('AuthorList', [])
            authors = ", ".join([
                f"{author.get('LastName', '')} {author.get('ForeName', '')}" 
                for author in author_list
            ])
            
            # Abstractは複数の部分に分かれていることがある
            abstract_parts = article.get('Abstract', {}).get('AbstractText', [])
            abstract = " ".join(abstract_parts)
            
            # 論文情報を辞書に格納
            article_info = {
                'pmid': pmid,
                'title': title,
                'pub_year': pub_year,
                'journal': journal,
                'authors': authors,
                'abstract': abstract,
                'url': f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/"
            }
            
            # Abstractを翻訳
            article_info['abstract_ja'] = translate_text(article_info['abstract'])
            
            # Slackに通知
            send_slack_notification(article_info)
            
            new_articles.append(article_info)
            sleep(1) # サーバーへの負荷を軽減

    if new_articles:
        print(f"\n{len(new_articles)}件の新規論文をCSVファイルに追加します。")
        df_new = pd.DataFrame(new_articles)
        
        df_all = pd.concat([df_existing, df_new], ignore_index=True)
        # UTF-8 with BOMで保存し、Excelでの文字化けを防ぐ
        df_all.to_csv(CSV_FILE_PATH, index=False, encoding='utf-8-sig')
        print(f"'{CSV_FILE_PATH}' に保存しました。")
    else:
        print("\n新しい論文は見つかりませんでした。")

# --- メイン処理を実行 ---
main_pubmed_search()

新規に論文リストを作成します。
PubMedで検索中... (検索語: (("llm"[Title/Abstract] OR "large language model"[Title/Abstract])) AND "medical"[Title/Abstract])
20件の論文情報を取得します。

--- 新規論文を処理中 (PMID: 40651009) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40442891) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40689366) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40689255) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40617017) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40581100) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40479778) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40706107) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40680485) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40633400) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40515934) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40708759) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40683982) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40677929) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40669284) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PMID: 40614511) ---
Slackに通知を送信しました。

--- 新規論文を処理中 (PM